In [1]:
import os
from os import getcwd
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import glob
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout,GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import skimage.io
from keras import backend as K


In [2]:
data_dir = Path('/content/SARS-CoV-2') # Dataset Location Path
train_dir = os.path.join(data_dir, 'Train')
val_dir = os.path.join(data_dir, 'Val')

train_cov_dir = os.path.join(train_dir, 'COVID') 
train_noncov_dir = os.path.join(train_dir, 'NON-COVID')
validation_cov_dir = os.path.join(val_dir, 'COVID')
validation_noncov_dir = os.path.join(val_dir, 'NON-COVID')

train_cov_fnames = os.listdir(train_cov_dir)
train_noncov_fnames = os.listdir(train_noncov_dir)
validation_cov_fnames = os.listdir(validation_cov_dir)
validation_noncov_fnames = os.listdir(validation_noncov_dir)

print("Train COVID Images: ", len(train_cov_fnames))
print("Train NON-COVID Images: ", len(train_noncov_fnames))
print("Val COVID Images: ", len(validation_cov_fnames))
print("Val NON-COVID Images: ", len(validation_noncov_fnames))


Train COVID Images:  4010
Train NON-COVID Images:  3935
Val COVID Images:  200
Val NON-COVID Images:  196


In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(224, 224)
                                                    )

validation_datagen = ImageDataGenerator(rescale=1./255)
 

validation_generator = validation_datagen.flow_from_directory(val_dir,
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              target_size=(224, 224))

Found 7945 images belonging to 2 classes.
Found 396 images belonging to 2 classes.


In [4]:
def mini_block(filters, x):
    conv = tf.keras.layers.Conv2D(filters , kernel_size=(3, 3),padding = "same")(x)
    bn = tf.keras.layers.BatchNormalization()(conv)
    act = tf.keras.layers.ReLU()(bn)
    return act

def double_mini_block(filters, x):
    t = mini_block(filters, x)
    t = mini_block(filters, t)
    return t

def MFL_Block(f,inp):
    block1 = mini_block(f,inp)
    block2 = double_mini_block(f,inp)
    c1 =  concatenate([block1, block2])
    conv = tf.keras.layers.Conv2D(f , kernel_size=(1, 1))(c1)
    block3 = mini_block(f,conv)
    block4 = mini_block(f,inp)
    c2 = concatenate([block3, block4])
    conv = tf.keras.layers.Conv2D(f, kernel_size=(1, 1))(c2)
    return conv

def MFL_Net():
    inp = tf.keras.layers.Input(shape=(224,224,3))
    b1 = mini_block(16,inp)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(b1)
    p1 = MFL_Block(16,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p1)
    p2 = MFL_Block(32,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p2)
    p3 = MFL_Block(64,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p3)
    p4 = MFL_Block(128,pool)
    globavg = tf.keras.layers.GlobalAveragePooling2D()(p4)
    drop = tf.keras.layers.Dropout(rate=0.5)(globavg)
    out = tf.keras.layers.Dense(2, activation='softmax')(drop)
    model = tf.keras.Model(inputs=inp, outputs=out)
    model.compile(optimizer = Adam(lr=0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
    return model


In [5]:
model = MFL_Net()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 224, 224, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 224, 224, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint('SARS-CoV-2_MFL_Net.hdf5', save_weights_only=True, save_best_only=True,mode='min'),
             tf.keras.callbacks.ReduceLROnPlateau( monitor = 'val_loss',factor=0.5, patience=4, min_lr=0.000000001)] 
history = model.fit_generator(train_generator,
            validation_data = validation_generator,
            epochs =70, verbose=1, callbacks = callbacks)